# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imported.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

## Make sure the initial dependencies and the Geoapify API key are imported

In [5]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps


# Import API key
from config import geoapify_key


In [7]:
# Configure gmaps
gmaps.configure (api_key=geoapify_key)

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [8]:
# Read the WeatherPy_vacation.csv into a DataFrame
file_path = "../Vacation_Search/WeatherPy_vacation.csv"
vacation_df = pd.read_csv(file_path)

# Display sample data
vacation_df.head(10)

,Unnamed: 0,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,2,Aksu,CN,82.42,clear sky,41.12,80.26,Pudong Holiday Hotel
1,1,7,Touros,BR,80.60,broken clouds,-5.20,-35.46,INN NEW HORIZON
2,2,10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
3,3,11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,39 On Nile Guest House
4,4,16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
5,5,18,Hyderabad,IN,87.01,haze,17.38,78.47,"Taj Krishna, Hyderabad"
6,6,20,Port Alfred,ZA,82.00,clear sky,-33.59,26.89,The Halyards Hotel
7,7,22,Atuona,PF,79.72,clear sky,-9.80,-139.03,Villa Enata
8,8,23,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
9,9,25,Nikolskoye,RU,88.00,clear sky,59.70,30.79,Tourist House - Sablino


In [9]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [10]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Bengkulu"]
vacation_end = vacation_start
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Meulaboh"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Banda Aceh"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Padang"]

In [12]:
# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = (vacation_start.to_numpy()[0][5], vacation_start.to_numpy()[0][6])
end = start
stop1 = (vacation_stop1.to_numpy()[0][5], vacation_stop1.to_numpy()[0][6])
stop2 = (vacation_stop2.to_numpy()[0][5], vacation_stop2.to_numpy()[0][6])
stop3 = (vacation_stop3.to_numpy()[0][5], vacation_stop3.to_numpy()[0][6])

In [14]:
# Define a new figure object
fig = gmaps.figure()

# Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints.
# The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
vacation_itinerary = gmaps.directions_layer(start, end, waypoints = [stop1, stop2, stop3], travel_mode = "DRIVING")

# Add the layer to the map
fig.add_layer(vacation_itinerary)

# Display the map
fig

TraitError: The 'start' trait of a Directions instance contains  which expected <gmaps.geotraitlets.Point object at 0x7faca1aadbe0>, not the str 'overcast clouds is not a valid latitude. Latitudes must be floats'.

In [15]:
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3
    ],
    ignore_index = True
)

# Display sample data
itinerary_df

,Unnamed: 0,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,80,197,Bengkulu,ID,78.84,overcast clouds,-3.80,102.27,Grage Hotel Bengkulu
1,119,268,Meulaboh,ID,76.66,overcast clouds,4.14,96.13,Kost Pria
2,160,352,Banda Aceh,ID,81.77,overcast clouds,5.56,95.32,OYO 884 Rumoh PMI Hotel
3,192,412,Padang,ID,79.16,scattered clouds,-0.95,100.35,Mercure Padang


In [16]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [17]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Define a new figure object
fig = gmaps.figure()

# Add the layer to the map
fig.add_layer(marker_layer)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))